In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import pathlib as pth
import importlib
import copy
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [2]:
spice_dir = pth.Path('/home/jleland/Data/external/spice/')
os.chdir(spice_dir)

In [3]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [96]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/angled_tiltscan_ext',
    'marconi/restart_test/sprobe_9hr_multirestart',
    'marconi/restart_test/sprobe_1hr',
    'marconi/restart_test/sprobe_1hr.1',
    'marconi/restart_test/sprobe_1hr_atrestart',
    'marconi/restart_test/sprobe_4hr_multirestart_r',
    'marconi/restart_test/sprobe_4hr_multirestart_2r',
    'marconi/restart_test/sprobe_4hr_multirestart_c',
    'marconi/restart_test/sprobe_4hr_multirestart_2c',
    'marconi/restart_test/sprobe_10hr_multirestart_r',
    'marconi/restart_test/sprobe_10hr_multirestart_2r',
    'marconi/restart_test/sprobe_control_mkomm',
    'marconi/restart_test/sprobe_20hr_multirestart_2.13_4c',
    'marconi/restart_test/sprobe_control',
    
    'marconi/restart_test/sprobe_4hr_multirestart_c',
    'marconi/restart_test/sprobe_4hr_multirestart_2c',
    'marconi/restart_test/sprobe_20hr_multirestart_mkomm_4c',
    'marconi/restart_test/sprobe_control_mkomm_rel',
    'marconi/restart_test/sprobe_20hr_multirestart_4c',
    'marconi/restart_test/sprobe_20hr_multirestart_4c.1',
    'marconi/restart_test/sprobe_20hr_multirestart_4c.2',
    'marconi/restart_test/sprobe_20hr_multirestart_4r',
    'marconi/restart_test/sprobe_20hr_multirestart_recomp_4c',
    'marconi/restart_test/sprobe_control_2.13',
}
single_sims = {
    'marconi/restart_test/sprobe_control_2.13_hdf5',
    'marconi/restart_test/sprobe_control_recomp',
    'marconi/restart_test/sprobe_control_2.13_',
    'marconi/restart_test/sprobe_20hr_multirestart_2.13_hdf5_4c'
}

In [97]:
scans_searchstr = '*/restart_test/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        print(f'Single sim {scan} found')
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
    print(f'{scan}: {all_run_dirs[scan]}\n')


Single sim marconi/restart_test/sprobe_20hr_multirestart_2.13_hdf5_4c found
marconi/restart_test/sprobe_20hr_multirestart_2.13_hdf5_4c: ['marconi/restart_test/sprobe_20hr_multirestart_2.13_hdf5_4c']

Single sim marconi/restart_test/sprobe_control_recomp found
marconi/restart_test/sprobe_control_recomp: ['marconi/restart_test/sprobe_control_recomp']

Single sim marconi/restart_test/sprobe_control_2.13_ found
marconi/restart_test/sprobe_control_2.13_: ['marconi/restart_test/sprobe_control_2.13_']

Single sim marconi/restart_test/sprobe_control_2.13_hdf5 found
marconi/restart_test/sprobe_control_2.13_hdf5: ['marconi/restart_test/sprobe_control_2.13_hdf5']



In [98]:
skippable = {}
scans = list(scans)

## Attempts to read and perhaps concatenate several Spice-2 hdf5 output files together 
The two scans selected above are runs using the newly updated spice-2.13 with the sprobe lowdens input file at 10$^\circ$ inclination 

In [99]:
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/restart_test/sprobe_20hr_multirestart_2.13_hdf5_4c
1 marconi/restart_test/sprobe_control_recomp
2 marconi/restart_test/sprobe_control_2.13_
3 marconi/restart_test/sprobe_control_2.13_hdf5


In [103]:
chosen_spl_path_old = spice_dir / scans[1]
chosen_spl_path = spice_dir / scans[2]

print(chosen_spl_path_old)
print(chosen_spl_path)

/home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_recomp
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_


In [104]:
# splopter = spl.Splopter(chosen_spl_path, reduce=desired_variables)
# splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)

In [108]:
# bu_folders = list(chosen_spl_path.glob('backup*'))
# chosen_spl_path_bu = chosen_spl_path / bu_folders[1]
# hdf5_files = list(chosen_spl_path_bu.glob('*t-*.h5'))

hdf5_files = list(chosen_spl_path.glob('*t-*.h5'))

for i, hfile in enumerate(hdf5_files):
    print(f'{i}) {hfile}')

0) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_.h5
1) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_00.h5
2) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_01.h5
3) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_02.h5
4) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_03.h5
5) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_04.h5
6) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_05.h5
7) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_06.h5
8) /home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_07.h5
9) /home/jleland/Data/external/spice/ma

## Closer look at the t-file 

In [109]:
print(hdf5_files[0])

/home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_/t-sprobe_control_2.13_.h5


In [110]:
tfile_name = hdf5_files[0]

tfile_ds = xr.open_dataset(tfile_name)
# tfile_ds = xr.open_dataset(tfile_name, engine='h5netcdf')

In [111]:
print(tfile_ds)
for dim, size in tfile_ds.dims.items():
    print(dim, size)

<xarray.Dataset>
Dimensions:            (phony_dim_0: 2000, phony_dim_1: 609, phony_dim_10: 10, phony_dim_11: 48, phony_dim_12: 1999, phony_dim_13: 336, phony_dim_14: 568525, phony_dim_15: 11, phony_dim_2: 193, phony_dim_3: 1, phony_dim_4: 379016, phony_dim_5: 2, phony_dim_6: 150, phony_dim_7: 117344, phony_dim_8: 4, phony_dim_9: 12)
Dimensions without coordinates: phony_dim_0, phony_dim_1, phony_dim_10, phony_dim_11, phony_dim_12, phony_dim_13, phony_dim_14, phony_dim_15, phony_dim_2, phony_dim_3, phony_dim_4, phony_dim_5, phony_dim_6, phony_dim_7, phony_dim_8, phony_dim_9
Data variables:
    Esct               (phony_dim_0) float64 ...
    Escy               (phony_dim_1, phony_dim_2) float64 ...
    Escz               (phony_dim_1, phony_dim_2) float64 ...
    Na                 (phony_dim_3) int32 ...
    Nc                 (phony_dim_3) int32 ...
    Np                 (phony_dim_3) int32 ...
    Npc                (phony_dim_3) int32 ...
    Ny                 (phony_dim_3) int32

In [112]:
rename_dims_dict = {
    'phony_dim_0': 'time',
    'phony_dim_1': 'y',
    'phony_dim_2': 'z',
    'phony_dim_3': 'sim',
    'phony_dim_4': 'diag_time',
    'phony_dim_5': 'species',
    'phony_dim_6': 'diag_hist_bin',
#     'phony_dim_7': ,
    'phony_dim_8': 'objects',
#     'phony_dim_9': ,
#     'phony_dim_10': ,
    'phony_dim_11': 'task',
    'phony_dim_12': 'objectsdiag_time',
#     'phony_dim_13': ,
    'phony_dim_14': 'timestep', 
#     'phony_dim_15': ,
}

tfile_ds = tfile_ds.rename_dims(rename_dims_dict)

In [113]:
print(tfile_ds)

<xarray.Dataset>
Dimensions:            (diag_hist_bin: 150, diag_time: 379016, objects: 4, objectsdiag_time: 1999, phony_dim_10: 10, phony_dim_13: 336, phony_dim_15: 11, phony_dim_7: 117344, phony_dim_9: 12, sim: 1, species: 2, task: 48, time: 2000, timestep: 568525, y: 609, z: 193)
Dimensions without coordinates: diag_hist_bin, diag_time, objects, objectsdiag_time, phony_dim_10, phony_dim_13, phony_dim_15, phony_dim_7, phony_dim_9, sim, species, task, time, timestep, y, z
Data variables:
    Esct               (time) float64 ...
    Escy               (y, z) float64 ...
    Escz               (y, z) float64 ...
    Na                 (sim) int32 ...
    Nc                 (sim) int32 ...
    Np                 (sim) int32 ...
    Npc                (sim) int32 ...
    Ny                 (sim) int32 ...
    Nz                 (sim) int32 ...
    Nzmax              (sim) int32 ...
    Pot                (y, z) float64 ...
    Potvac             (y, z) float64 ...
    ProbePot          

In [114]:
currents = tfile_ds[['objectscurrenti', 'objectscurrente']].sel(objects=slice(2,4)).sum('objects')

total_current = xr.concat([currents['objectscurrenti'], currents['objectscurrente']], 'species').sum('species')
total_current.plot()

In [123]:
fig, axes = plt.subplots(2, sharex=True)
print(tfile_ds.load()[['Na', 'Nc', 'Np', 'Ny', 'Nz']].values)

probe_pot = tfile_ds['ProbePot'] #.sel(diag_time=slice(None, None, 100))

print(len(total_current), len(probe_pot))

axes[0].plot(tfile_ds['t'].values[:-1], total_current)
xr.concat([currents['objectscurrenti'], currents['objectscurrente']], 'species').sum('species').plot(ax=axes[1])

axes[1].set_ylim(-10, 40)

<bound method Mapping.values of <xarray.Dataset>
Dimensions:  (sim: 1)
Dimensions without coordinates: sim
Data variables:
    Na       (sim) int32 379016
    Nc       (sim) int32 189508
    Np       (sim) int32 568524
    Ny       (sim) int32 609
    Nz       (sim) int32 193>
1999 379016


(-10, 40)

In [44]:
tfile_ds['objectsenum'].astype(np.int64).plot(x='y', y='z')
plt.axis('equal')

(-0.5, 608.5, -0.5, 192.5)

In [46]:
tfile_ds['rho01'].plot()

## Closer look at the particle files

In [31]:
pfile_name = hdf5_files[48]
print(pfile_name)

pfile_ds = xr.open_dataset(pfile_name)
# tfile_ds = xr.open_dataset(tfile_name, engine='h5netcdf')

/home/jleland/Data/external/spice/marconi/restart_test/sprobe_control_2.13_hdf5/t-sprobe_control_2.13_hdf547.h5


In [32]:
pfile_ds = pfile_ds.load()

In [33]:
print(pfile_ds)
for dim, size in pfile_ds.dims.items():
    print(dim, size)

<xarray.Dataset>
Dimensions:  (phony_dim_0: 1, phony_dim_1: 15153)
Dimensions without coordinates: phony_dim_0, phony_dim_1
Data variables:
    count    (phony_dim_0) int32 568524
    nproc    (phony_dim_0) int32 47
    ntot     (phony_dim_0) int32 15153
    stype    (phony_dim_1) int32 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    ux       (phony_dim_1) float64 -18.02 -0.4697 4.11 ... -26.81 -18.11 -16.52
    uy       (phony_dim_1) float64 30.17 -13.72 -20.34 ... 20.01 47.47 27.88
    uz       (phony_dim_1) float64 -0.6151 -0.8843 1.762 ... 15.87 15.56 14.22
    version  (phony_dim_0) int32 50
    y        (phony_dim_1) float64 564.0 553.0 540.0 592.3 ... 566.1 564.4 572.5
    z        (phony_dim_1) float64 280.1 280.6 280.9 280.9 ... 280.0 280.0 280.0
phony_dim_0 1
phony_dim_1 15153


## Comparing the old and new versions with the control runs in each case

In [125]:
def plot_raw_data_splopter(path_to_splopter, axes=None, label=None):
    if label is None:
        label = str(path_to_splopter).split('/')[-1]
    print(label)
    
    splopter = spl.Splopter(path_to_splopter, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)

    # Extract relevant arrays from the matlab file
    probe_indices = splopter.parser.get_probe_obj_indices()
    probe_current_e = 0.0
    probe_current_i = 0.0

    time = np.squeeze(splopter.tdata.t)[:-1]
    for index in probe_indices:
        probe_current_e += np.squeeze(splopter.tdata.objectscurrente)[index]
        probe_current_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
    probe_bias = np.squeeze(splopter.tdata.diagnostics['ProbePot'])
    probe_current_tot = probe_current_i + probe_current_e
    
    print(splopter.tdata.diagnostics)
    
    if axes is None:
        fig, axes = plt.subplots(2)
    
    axes[0].set_title(label)
    axes[0].plot(probe_bias, label=label)
#     axes[0].plot(universal_probe_bias)
    axes[0].legend()
    axes[1].plot(time, probe_current_tot, label=label)
#     axes[1].plot(time, probe_current_i)
#     axes[1].plot(time, probe_current_e)
    axes[1].legend()
    
    return splopter

In [128]:
fig, axes = plt.subplots(1, 2)

splopter = plot_raw_data_splopter(chosen_spl_path_old, axes=axes, label='vers. 2.11')

axes[0].plot(probe_pot, label='vers. 2.13')
axes[0].set_title('Probe bias diagnostic')
axes[0].set_ylabel(r'$\hat{V}$')
axes[0].set_xlabel(r'arb units')
axes[0].legend()

axes[1].plot(tfile_ds['t'].values[:-1], total_current, label='vers. 2.13')
axes[1].set_title('Combined objectscurrent for probe objects')
axes[1].set_ylabel(r'$\hat{I}$')
axes[1].set_xlabel(r'$\hat{t}$')
axes[1].set_ylim(-350, 50)


axes[1].legend()

vers. 2.11
{'ProbePot': array([[-9.9 ],
       [-9.9 ],
       [-9.9 ],
       ...,
       [10.05],
       [10.05],
       [10.05]])}
